<a href="https://colab.research.google.com/github/SulemanShahani/Similarity-Search-on-Reuters-news-Data-Set-from-Hugging-Face/blob/main/Similarity_Search_on_Reuters_news_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM

import faiss

# Load the Reuters news dataset from the Hugging Face hub
from langchain.document_loaders import HuggingFaceDatasetLoader

loader = HuggingFaceDatasetLoader("reuters21578", page_content_column="text", name="ModLewis")
data = loader.load()

# Splitting the data into chunks (only using top 500 articles for simplicity)
top_500_articles = data[:500]

# Initialize the RecursiveCharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=128, chunk_overlap=0)

# Split documents into chunks
docs = splitter.split_documents(top_500_articles)
text_list = [doc.page_content for doc in docs]

# Load the tokenizer and embedding model
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-small-en-v1.5")
model = AutoModel.from_pretrained("BAAI/bge-small-en-v1.5")

# Load the tokenizer and generation model
generation_tokenizer = AutoTokenizer.from_pretrained("t5-small")
generation_model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

# Function to create embeddings
def create_embeddings(texts):
    embeddings = []
    for text in texts:
        tokens = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
        with torch.no_grad():
            outputs = model(**tokens)
        embeddings.append(outputs.last_hidden_state.mean(dim=1).numpy())
    return np.vstack(embeddings)

# Create embeddings for documents
document_embeddings = create_embeddings(text_list)

# Create FAISS index
index = faiss.IndexFlatL2(document_embeddings.shape[1])
index.add(document_embeddings)

# Function to generate query embedding
def generate_query_embedding(query):
    tokens = tokenizer(query, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**tokens)
    return outputs.last_hidden_state.mean(dim=1).numpy()

# Function to retrieve documents
def retrieve_documents(query_embedding, k=5):
    distances, indices = index.search(query_embedding, k)
    return [text_list[i] for i in indices[0]]

# Function to generate response
def generate_response(query, retrieved_docs):
    context = " ".join(retrieved_docs)
    input_text = f"question: {query} context: {context}"
    input_ids = generation_tokenizer.encode(input_text, return_tensors="pt", truncation=True)
    outputs = generation_model.generate(input_ids, max_length=100)
    return generation_tokenizer.decode(outputs[0], skip_special_tokens=True)

# Interactive function
def interact():
    while True:
        query = input("Enter your question (or type 'quit' to exit): ")
        if query.lower() == "quit":
            break
        query_embedding = generate_query_embedding(query)
        retrieved_docs = retrieve_documents(query_embedding)
        response = generate_response(query, retrieved_docs)
        print(f"Response: {response}")

# Run interaction
interact()


/usr/local/lib/python3.10/dist-packages/datasets/load.py:2554: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for reuters21578 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/reuters21578.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating test split:   0%|          | 0/6188 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/13625 [00:00<?, ? examples/s]

Generating unused split:   0%|          | 0/722 [00:00<?, ? examples/s]

...

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Enter your question (or type 'quit' to exit): 
Response: Neste Oy, Celanese CanadanInc, Hoechst Celanese Corporation and in the March 31 second quarter
Enter your question (or type 'quit' to exit): japnese
Response: Japan to the General Agreement onnTariffs and Trade last year
Enter your question (or type 'quit' to exit): Ukraine
Response: Ukraine
Enter your question (or type 'quit' to exit): trade
Response: imports and settle outstanding trade issues
Enter your question (or type 'quit' to exit): Japan
Response: Japan has until the next meeting of senior officials fromnthe two countries in May or June to come up with a Japan's electric powernin the fiscal year ended March 31, supplying an estimated 27npct on a kilowatt/hour basis
Enter your question (or type 'quit' to exit): 
Response: Neste Oy, Celanese CanadanInc, Hoechst Celanese Corporation and in the March 31 second quarter
Enter your question (or type 'quit' to exit): Holand gdp
Response: The report said Thailand benefited from t

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from sklearn.metrics.pairwise import cosine_similarity

# Function to evaluate system using quantitative metrics
def evaluate_system(test_queries, expected_responses):
    bleu_scores = []
    cosine_similarities = []

    for query, expected_response in zip(test_queries, expected_responses):
        query_embedding = generate_query_embedding(query)
        retrieved_docs = retrieve_documents(query_embedding)
        generated_response = generate_response(query, retrieved_docs)

        # Calculate BLEU score
        bleu_score = sentence_bleu([expected_response.split()], generated_response.split(), smoothing_function=SmoothingFunction().method1)
        bleu_scores.append(bleu_score)

        # Calculate cosine similarity between expected and generated responses
        expected_embedding = generate_query_embedding(expected_response)
        generated_embedding = generate_query_embedding(generated_response)
        cosine_sim = cosine_similarity(expected_embedding.reshape(1, -1), generated_embedding.reshape(1, -1))[0][0]
        cosine_similarities.append(cosine_sim)

        # Print the results for inspection
        print(f"Query: {query}")
        print(f"Generated Response: {generated_response}")
        print(f"Expected Response: {expected_response}")
        print(f"BLEU Score: {bleu_score}")
        print(f"Cosine Similarity: {cosine_sim}")
        print("=" * 50)

    # Print average scores
    print(f"Average BLEU Score: {np.mean(bleu_scores)}")
    print(f"Average Cosine Similarity: {np.mean(cosine_similarities)}")

# Test queries and expected responses for evaluation
test_queries = [
    "What are the latest developments in technology?",
    "How has the stock market performed this week?",
    "What are the implications of climate change on agriculture?",
    "Explain the impact of recent geopolitical events on global trade.",
    "How has the COVID-19 pandemic affected the travel industry?"
]

expected_responses = [
    "Latest developments in technology include advancements in AI and robotics.",
    "The stock market has shown fluctuations this week due to economic uncertainty.",
    "Climate change poses significant challenges for agriculture, affecting crop yields and water availability.",
    "Recent geopolitical events have led to fluctuations in global trade, impacting economies worldwide.",
    "The travel industry has been severely affected by the COVID-19 pandemic, leading to travel restrictions and reduced tourism."
]

# Evaluate the system
evaluate_system(test_queries, expected_responses)

# Run interaction
interact()

Query: What are the latest developments in technology?
Generated Response: television and audio equipment and computers
Expected Response: Latest developments in technology include advancements in AI and robotics.
BLEU Score: 0.020960166113993737
Cosine Similarity: 0.5676347017288208
Query: How has the stock market performed this week?
Generated Response: rose sharply
Expected Response: The stock market has shown fluctuations this week due to economic uncertainty.
BLEU Score: 0
Cosine Similarity: 0.5019999742507935
Query: What are the implications of climate change on agriculture?
Generated Response: a greater role innstimulating economic growth
Expected Response: Climate change poses significant challenges for agriculture, affecting crop yields and water availability.
BLEU Score: 0
Cosine Similarity: 0.5311249494552612
Query: Explain the impact of recent geopolitical events on global trade.
Generated Response: trade dispute between the United States and Japan
Expected Response: Recent

In [2]:
 #!pip install faiss-cpu
!pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.3 MB/s eta 0:00:00


In [6]:

!pip install langchain
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting u